In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import randint, norm
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [4]:
spotify = pd.read_csv('./assets/spotify_songs.csv')

spotify["is_popular"] = spotify["track_popularity"] > 80

spotify = spotify.drop(columns=["track_id", "track_name", "track_artist", "track_album_id", "track_album_name", "playlist_name", "playlist_id"])

spotify = pd.get_dummies(data=spotify, columns=["playlist_genre", "playlist_subgenre"])

spotify["track_album_release_date"] = spotify["track_album_release_date"].str.split("-").str[0]

spotify["track_album_release_date"] = spotify["track_album_release_date"].astype(float)

spotify.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32821 entries, 0 to 32820
Data columns (total 45 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   track_popularity                             32821 non-null  int64  
 1   track_album_release_date                     32821 non-null  float64
 2   danceability                                 32821 non-null  float64
 3   energy                                       32821 non-null  float64
 4   key                                          32821 non-null  int64  
 5   loudness                                     32821 non-null  float64
 6   mode                                         32821 non-null  int64  
 7   speechiness                                  32821 non-null  float64
 8   acousticness                                 32821 non-null  float64
 9   instrumentalness                             32821 non-null  float64
 10

In [5]:
X = spotify.drop(columns=['track_popularity', 'is_popular'])
y = spotify['is_popular']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [6]:
forest = RandomForestClassifier(random_state=42)
forest.fit(X_train, y_train)
forest.score(X_test, y_test)

predicted_labels = forest.predict(X_test)

confusion_matrix(y_test, predicted_labels)

array([[6289,    8],
       [ 198,   70]], dtype=int64)

In [13]:
#Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 10)]
#Number of features to consider at every split
max_features = ['auto', 'sqrt']
#Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

forest_cv = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions= {'n_estimators': n_estimators,
                          'max_features': max_features,
                          'class_weight': [{0: 1, 1: 2}, {0: 1, 1: 5}],
                          'max_depth': max_depth},
    n_jobs=-1,
    cv=5,
    verbose=999,
    scoring='f1',
    n_iter=25,
)

forest_cv.fit(X_train, y_train)
forest_cv.score(X_test, y_test)

predicted_labels = forest_cv.predict(X_test)

confusion_matrix(y_test, predicted_labels)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


C:\Users\benni\miniconda3\envs\dsai\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
45 fits failed out of a total of 125.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
31 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\benni\miniconda3\envs\dsai\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\benni\miniconda3\envs\dsai\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\benni\miniconda3\envs\dsai\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File

array([[6289,    8],
       [ 177,   91]], dtype=int64)